In [1]:
# Importing the required modules and sub-modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib notebook
sns.set()

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.mixture import GaussianMixture

from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [2]:
# Loading in the data and replicating for redundancy

def import_data():
    train_data = pd.read_csv('Train.csv')
    test_data = pd.read_csv('Test.csv')
    
    train = train_data.copy()
    test = test_data.copy()
    
    return train, test

In [3]:
train, test = import_data()

In [4]:
# Peeking at the data

train.head(10)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.2820,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.0000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.0000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
5,Apcnt_1000020,3724.0,NaN,0.0000,0.0,0.0000,192944.0,9244585.0,6761209.0,67963357.0,...,0.030558,12.12,0.382960,NaN,0.0,lending,NaN,0.000000,NaN,no
6,Apcnt_1000024,3350.0,0.24205,0.3132,0.0,2.8188,NaN,580388.0,NaN,96472.0,...,0.660000,1.01,0.513504,NaN,0.0,charge,NaN,2.146512,0.004390,yes
7,Apcnt_1000028,3416.0,0.87020,0.0570,0.0,0.0000,17505.0,186331.0,186331.0,1484813.0,...,0.137500,3.03,0.635960,NaN,0.0,charge,155.295994,0.946074,0.128389,no
8,Apcnt_1000036,3376.0,0.29180,0.5586,0.0,0.0000,18283.0,1105149.0,18283.0,115533.0,...,0.471427,4.04,0.410880,0.0,0.0,charge,NaN,0.641601,0.009990,no
9,Apcnt_1000040,3310.0,0.24395,0.1562,0.0,0.0000,NaN,0.0,NaN,NaN,...,0.825000,1.01,0.800000,NaN,NaN,charge,NaN,0.000000,NaN,yes


In [5]:
test.head(10)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124
5,Apcnt_1000084,NaN,0.41005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,charge,NaN,NaN,NaN
6,Apcnt_1000104,3408.0,0.74605,0.6064,0.0000,0.0,0.0,3769410.0,12059.0,1230018.0,...,NaN,0.061116,12.12,0.642296,0.0,0.0,charge,221.047330,1.245020,0.074203
7,Apcnt_1000116,3296.0,0.41745,0.9796,0.0000,0.0,NaN,56794.0,NaN,NaN,...,NaN,1.100000,1.01,0.532248,NaN,NaN,charge,NaN,0.000000,NaN
8,Apcnt_1000128,3424.0,0.17380,0.1040,0.0000,0.0,0.0,2317662.0,1467308.0,55619609.0,...,NaN,0.000000,12.12,0.483792,0.0,0.0,charge,1464.871576,3.623188,1.154236
9,Apcnt_1000156,3408.0,0.20105,0.0000,0.0000,0.0,0.0,911427.0,389389.0,3613032.0,...,NaN,0.194117,12.12,0.760912,0.0,0.0,charge,444.468343,4.000000,0.037489


In [6]:
# Data description

train_tr = np.transpose(train.describe())

In [7]:
test_tr = np.transpose(test.describe())

In [8]:
train_tr

,count,mean,std,min,25%,50%,75%,max
form_field1,53471.0,3.491796e+03,1.884624e+02,2990.000000,3.358000e+03,3.484000e+03,3.620000e+03,3.900000e+03
form_field2,52156.0,5.507368e-01,8.209792e-01,0.000000,7.078750e-02,2.675750e-01,7.195125e-01,1.801505e+01
form_field3,55645.0,1.052225e+00,2.147768e+00,0.000000,0.000000e+00,6.200000e-02,1.282000e+00,5.737160e+01
form_field4,55645.0,8.519789e-01,3.157692e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,9.167220e+01
form_field5,55645.0,1.956317e+00,1.051240e+01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.077486e+02
form_field6,42640.0,6.244479e+05,1.433422e+06,0.000000,1.400400e+04,1.155330e+05,5.259280e+05,5.313546e+07
form_field7,50837.0,6.865210e+06,1.912729e+07,0.000000,6.869740e+05,2.704328e+06,6.993831e+06,2.158794e+09
form_field8,42640.0,2.626690e+06,3.927355e+06,0.000000,1.929440e+05,9.639420e+05,3.751516e+06,1.037397e+08
form_field9,47992.0,1.316002e+07,1.977963e+07,0.000000,1.368502e+06,5.506295e+06,1.694552e+07,3.200533e+08
form_field10,55645.0,1.185585e+07,2.669459e+07,0.000000,4.500730e+05,3.707559e+06,1.326101e+07,2.191361e+09


In [9]:
test_tr

,count,mean,std,min,25%,50%,75%,max
form_field1,22890.0,3.492284e+03,1.905028e+02,2.986000e+03,3.356000e+03,3.484000e+03,3.624000e+03,3.900000e+03
form_field2,22291.0,5.576761e-01,8.265431e-01,0.000000e+00,6.867500e-02,2.732500e-01,7.288500e-01,2.231505e+01
form_field3,23854.0,1.065443e+00,2.198444e+00,0.000000e+00,0.000000e+00,5.820000e-02,1.304250e+00,3.454140e+01
form_field4,23854.0,8.591462e-01,3.403115e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.064528e+02
form_field5,23854.0,2.183538e+00,1.141571e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.978856e+02
form_field6,18396.0,6.263036e+05,1.457540e+06,0.000000e+00,1.400400e+04,1.155330e+05,5.159112e+05,4.818738e+07
form_field7,21769.0,6.797033e+06,1.626022e+07,0.000000e+00,6.725810e+05,2.719888e+06,7.073576e+06,7.709887e+08
form_field8,18396.0,2.654142e+06,3.968185e+06,0.000000e+00,1.816630e+05,9.594685e+05,3.799849e+06,1.135141e+08
form_field9,20600.0,1.350593e+07,2.289125e+07,0.000000e+00,1.349441e+06,5.529830e+06,1.728658e+07,1.443921e+09
form_field10,23853.0,1.187478e+07,2.477113e+07,0.000000e+00,4.208980e+05,3.651543e+06,1.318126e+07,7.741014e+08


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Applicant_ID  24000 non-null  object 
 1   form_field1   22890 non-null  float64
 2   form_field2   22291 non-null  float64
 3   form_field3   23854 non-null  float64
 4   form_field4   23854 non-null  float64
 5   form_field5   23854 non-null  float64
 6   form_field6   18396 non-null  float64
 7   form_field7   21769 non-null  float64
 8   form_field8   18396 non-null  float64
 9   form_field9   20600 non-null  float64
 10  form_field10  23853 non-null  float64
 11  form_field11  10602 non-null  float64
 12  form_field12  19817 non-null  float64
 13  form_field13  21537 non-null  float64
 14  form_field14  24000 non-null  int64  
 15  form_field15  14408 non-null  float64
 16  form_field16  18526 non-null  float64
 17  form_field17  19305 non-null  float64
 18  form_field18  19631 non-nu

In [12]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [13]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [14]:
train_var = pd.DataFrame({'unique' : train.nunique(),
                          'unique_%' : 100*train.nunique()/train.shape[0],
                          'null' : train.isnull().sum(),
                          'null_%' : 100*train.isnull().sum()/train.shape[0],
                          'dtype' : train.dtypes})
train_var

,unique,unique_%,null,null_%,dtype
Applicant_ID,56000,100.000000,0,0.000000,object
form_field1,449,0.801786,2529,4.516071,float64
form_field2,20015,35.741071,3844,6.864286,float64
form_field3,14617,26.101786,355,0.633929,float64
form_field4,2927,5.226786,355,0.633929,float64
form_field5,2991,5.341071,355,0.633929,float64
form_field6,5846,10.439286,13360,23.857143,float64
form_field7,23199,41.426786,5163,9.219643,float64
form_field8,12820,22.892857,13360,23.857143,float64
form_field9,22650,40.446429,8008,14.300000,float64


In [15]:
test_var = pd.DataFrame({'unique' : test.nunique(),
                         'unique_%' : 100*test.nunique()/test.shape[0],
                         'null' : test.isnull().sum(),
                         'null_%' : 100*test.isnull().sum()/test.shape[0],
                         'dtype' : test.dtypes})
test_var

,unique,unique_%,null,null_%,dtype
Applicant_ID,24000,100.000000,0,0.000000,object
form_field1,440,1.833333,1110,4.625000,float64
form_field2,11473,47.804167,1709,7.120833,float64
form_field3,7847,32.695833,146,0.608333,float64
form_field4,1578,6.575000,146,0.608333,float64
form_field5,1560,6.500000,146,0.608333,float64
form_field6,3567,14.862500,5604,23.350000,float64
form_field7,12967,54.029167,2231,9.295833,float64
form_field8,7360,30.666667,5604,23.350000,float64
form_field9,11706,48.775000,3400,14.166667,float64


In [16]:
# Encoding categorical features

train['form_field47'] = pd.get_dummies(train[['form_field47']])
test['form_field47'] = pd.get_dummies(test[['form_field47']])

In [17]:
train['form_field47'].unique()

array([1, 0], dtype=uint8)

In [18]:
test['form_field47'].unique()

array([1, 0], dtype=uint8)

In [19]:
np.transpose(train.head())

,0,1,2,3,4
Applicant_ID,Apcnt_1000000,Apcnt_1000004,Apcnt_1000008,Apcnt_1000012,Apcnt_1000016
form_field1,3436,3456,3276,3372,3370
form_field2,0.28505,0.674,0.53845,0.17005,0.7727
form_field3,1.656,0.2342,3.151,0.505,1.101
form_field4,0,0,0,0,0
form_field5,0,0,6.282,0,0
form_field6,0,0,NaN,192166,1556
form_field7,1.06897e+07,898979,956940,3.0447e+06,214728
form_field8,252072,497531,NaN,385499,214728
form_field9,4.27278e+06,9.07381e+06,192944,3.98647e+06,1.28409e+06


In [20]:
np.transpose(test.head())

,0,1,2,3,4
Applicant_ID,Apcnt_1000032,Apcnt_1000048,Apcnt_1000052,Apcnt_1000076,Apcnt_1000080
form_field1,3236,3284,NaN,3232,3466
form_field2,0.34875,1.2736,0.27505,0.28505,2.09545
form_field3,10.2006,2.9606,0.06,2.8032,0.8318
form_field4,0,9.0198,0,0,2.5182
form_field5,0,0,0,0,0
form_field6,418564,0,NaN,0,19839
form_field7,418564,9.85882e+06,NaN,473802,1.15066e+06
form_field8,418564,49014,NaN,473802,1.15066e+06
form_field9,540710,1.5101e+06,NaN,1.72444e+06,7.86052e+06


In [21]:
for column in ['form_field16', 'form_field17', 'form_field18']:
    print(test[column].value_counts())

0.0    15453
1.0     2227
2.0      577
3.0      175
4.0       57
5.0       22
6.0       11
7.0        3
9.0        1
Name: form_field16, dtype: int64
0.0     15860
1.0      2389
2.0       662
3.0       236
4.0        93
5.0        37
6.0        13
7.0        12
10.0        1
9.0         1
8.0         1
Name: form_field17, dtype: int64
0.0     16060
1.0      2481
2.0       685
3.0       243
4.0        96
5.0        33
6.0        17
7.0        13
10.0        1
9.0         1
8.0         1
Name: form_field18, dtype: int64


In [22]:
# Initializing required estimators and transformers

labelEncoder = LabelEncoder()

In [23]:
# Encoding class label

train['default_status'] = labelEncoder.fit_transform(train['default_status'])

In [24]:
# Splitting data into features and targets
class_weight = {0 : 1,
                1 : int(train.default_status.value_counts()[0]/train.default_status.value_counts()[1])}
X = train.drop(['Applicant_ID', 'default_status'], axis = 1).values
y = train['default_status'].values

In [25]:
modelA, modelB, modelC, modelD, modelE, modelF = [AdaBoostClassifier(random_state = 1),
                                                  
                                                  RandomForestClassifier(n_estimators = 200,
                                                                         criterion = 'entropy',
                                                                         random_state = 1),
                                                  
                                                  ExtraTreesClassifier(criterion = 'entropy',
                                                                       random_state = 1),
                                                  
                                                  LGBMClassifier(n_estimators = 1000,
                                                                 class_weight = class_weight,
                                                                 random_state = 1),
                                                  
                                                  CatBoostClassifier(learning_rate = 0.01,
                                                                     iterations = 1700,
                                                                     l2_leaf_reg = 3.5,
                                                                     depth = 8,
                                                                     rsm = 0.981,
                                                                     random_state = 1),
                                                  
                                                  GradientBoostingClassifier(n_estimators = 300,
                                                                             random_state = 1)]

In [26]:
# Stating the evaluation procedure for model
def evaluate_model(model, X, y):
    
    cv = RepeatedStratifiedKFold(random_state = 1, n_repeats = 3, n_splits = 10)
    scores = cross_val_score(model, X, y, cv = cv, scoring = 'roc_auc', n_jobs = -1)
    return 100*np.mean(scores), 100*np.std(scores)

# Evaluating different models...

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
from sklearn.metrics import roc_auc_score

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, stratify = y, random_state =1)

In [30]:
def train_model(model, X_train, y_train, X_test, y_test):
    
    for strategy in strategies:
        imputer = SimpleImputer(strategy = strategy)
        scaler = MinMaxScaler()
        power = PowerTransformer(method = 'box-cox', standardize = True)
        
        steps = [('imputer', imputer),
                 ('scaler', scaler),
                 ('power', power),
                 ('model', model)]
        
        pipeline = Pipeline(steps = steps)
        model.fit(X_train, y_train)
        ygnat = model.predict_proba(X_test)
        score = roc_auc_score(y_test, ygnat[:, 1])
        
        print('{} {} Accuracy: {}'.format(model, strategy.capitalize(), score))

In [33]:
train_model(modelE, X_train, y_train, X_test, y_test)

0:	learn: 0.6869323	total: 216ms	remaining: 3m 35s
1:	learn: 0.6808218	total: 276ms	remaining: 2m 17s
2:	learn: 0.6744626	total: 334ms	remaining: 1m 50s
3:	learn: 0.6682529	total: 390ms	remaining: 1m 37s
4:	learn: 0.6621438	total: 461ms	remaining: 1m 31s
5:	learn: 0.6564608	total: 522ms	remaining: 1m 26s
6:	learn: 0.6512542	total: 575ms	remaining: 1m 21s
7:	learn: 0.6456735	total: 632ms	remaining: 1m 18s
8:	learn: 0.6405924	total: 698ms	remaining: 1m 16s
9:	learn: 0.6355482	total: 768ms	remaining: 1m 16s
10:	learn: 0.6310475	total: 847ms	remaining: 1m 16s
11:	learn: 0.6264590	total: 915ms	remaining: 1m 15s
12:	learn: 0.6215118	total: 988ms	remaining: 1m 15s
13:	learn: 0.6169042	total: 1.05s	remaining: 1m 13s
14:	learn: 0.6125261	total: 1.1s	remaining: 1m 12s
15:	learn: 0.6086055	total: 1.18s	remaining: 1m 12s
16:	learn: 0.6042493	total: 1.24s	remaining: 1m 11s
17:	learn: 0.6001476	total: 1.29s	remaining: 1m 10s
18:	learn: 0.5963669	total: 1.37s	remaining: 1m 10s
19:	learn: 0.5923631	to

162:	learn: 0.4304004	total: 9.74s	remaining: 50s
163:	learn: 0.4301843	total: 9.8s	remaining: 49.9s
164:	learn: 0.4299398	total: 9.85s	remaining: 49.9s
165:	learn: 0.4296659	total: 9.91s	remaining: 49.8s
166:	learn: 0.4294214	total: 9.97s	remaining: 49.7s
167:	learn: 0.4291619	total: 10s	remaining: 49.6s
168:	learn: 0.4289446	total: 10.1s	remaining: 49.5s
169:	learn: 0.4287565	total: 10.1s	remaining: 49.4s
170:	learn: 0.4284993	total: 10.2s	remaining: 49.4s
171:	learn: 0.4282828	total: 10.2s	remaining: 49.3s
172:	learn: 0.4280619	total: 10.3s	remaining: 49.2s
173:	learn: 0.4278281	total: 10.3s	remaining: 49.1s
174:	learn: 0.4275991	total: 10.4s	remaining: 49.1s
175:	learn: 0.4274186	total: 10.5s	remaining: 49s
176:	learn: 0.4272191	total: 10.5s	remaining: 48.9s
177:	learn: 0.4270172	total: 10.6s	remaining: 48.8s
178:	learn: 0.4268420	total: 10.6s	remaining: 48.8s
179:	learn: 0.4266237	total: 10.7s	remaining: 48.7s
180:	learn: 0.4264138	total: 10.7s	remaining: 48.6s
181:	learn: 0.42625

324:	learn: 0.4109427	total: 18.7s	remaining: 38.8s
325:	learn: 0.4108863	total: 18.8s	remaining: 38.8s
326:	learn: 0.4108385	total: 18.8s	remaining: 38.7s
327:	learn: 0.4107665	total: 18.9s	remaining: 38.6s
328:	learn: 0.4107146	total: 18.9s	remaining: 38.6s
329:	learn: 0.4106667	total: 19s	remaining: 38.5s
330:	learn: 0.4106147	total: 19s	remaining: 38.5s
331:	learn: 0.4105491	total: 19.1s	remaining: 38.4s
332:	learn: 0.4104871	total: 19.1s	remaining: 38.4s
333:	learn: 0.4104321	total: 19.2s	remaining: 38.3s
334:	learn: 0.4103792	total: 19.2s	remaining: 38.2s
335:	learn: 0.4103191	total: 19.3s	remaining: 38.1s
336:	learn: 0.4102510	total: 19.4s	remaining: 38.1s
337:	learn: 0.4102012	total: 19.4s	remaining: 38s
338:	learn: 0.4101462	total: 19.5s	remaining: 38s
339:	learn: 0.4100929	total: 19.5s	remaining: 37.9s
340:	learn: 0.4100313	total: 19.6s	remaining: 37.8s
341:	learn: 0.4099754	total: 19.6s	remaining: 37.8s
342:	learn: 0.4099122	total: 19.7s	remaining: 37.7s
343:	learn: 0.409858

484:	learn: 0.4035636	total: 27.4s	remaining: 29.1s
485:	learn: 0.4035284	total: 27.5s	remaining: 29.1s
486:	learn: 0.4034886	total: 27.5s	remaining: 29s
487:	learn: 0.4034473	total: 27.6s	remaining: 28.9s
488:	learn: 0.4034187	total: 27.6s	remaining: 28.9s
489:	learn: 0.4033792	total: 27.7s	remaining: 28.8s
490:	learn: 0.4033366	total: 27.8s	remaining: 28.8s
491:	learn: 0.4033000	total: 27.8s	remaining: 28.7s
492:	learn: 0.4032618	total: 27.9s	remaining: 28.7s
493:	learn: 0.4032345	total: 27.9s	remaining: 28.6s
494:	learn: 0.4032053	total: 28s	remaining: 28.5s
495:	learn: 0.4031677	total: 28s	remaining: 28.5s
496:	learn: 0.4031459	total: 28.1s	remaining: 28.4s
497:	learn: 0.4031090	total: 28.1s	remaining: 28.4s
498:	learn: 0.4030732	total: 28.2s	remaining: 28.3s
499:	learn: 0.4030306	total: 28.3s	remaining: 28.3s
500:	learn: 0.4029886	total: 28.3s	remaining: 28.2s
501:	learn: 0.4029529	total: 28.4s	remaining: 28.1s
502:	learn: 0.4029134	total: 28.4s	remaining: 28.1s
503:	learn: 0.4028

646:	learn: 0.3984036	total: 36.3s	remaining: 19.8s
647:	learn: 0.3983673	total: 36.3s	remaining: 19.7s
648:	learn: 0.3983501	total: 36.4s	remaining: 19.7s
649:	learn: 0.3983222	total: 36.4s	remaining: 19.6s
650:	learn: 0.3983015	total: 36.5s	remaining: 19.6s
651:	learn: 0.3982708	total: 36.5s	remaining: 19.5s
652:	learn: 0.3982435	total: 36.6s	remaining: 19.5s
653:	learn: 0.3982222	total: 36.7s	remaining: 19.4s
654:	learn: 0.3981934	total: 36.7s	remaining: 19.3s
655:	learn: 0.3981529	total: 36.8s	remaining: 19.3s
656:	learn: 0.3981160	total: 36.8s	remaining: 19.2s
657:	learn: 0.3980898	total: 36.9s	remaining: 19.2s
658:	learn: 0.3980606	total: 36.9s	remaining: 19.1s
659:	learn: 0.3980438	total: 37s	remaining: 19.1s
660:	learn: 0.3980128	total: 37.1s	remaining: 19s
661:	learn: 0.3979899	total: 37.1s	remaining: 19s
662:	learn: 0.3979563	total: 37.2s	remaining: 18.9s
663:	learn: 0.3979390	total: 37.2s	remaining: 18.8s
664:	learn: 0.3979139	total: 37.3s	remaining: 18.8s
665:	learn: 0.3978

...

208:	learn: 0.4217213	total: 12.8s	remaining: 48.3s
209:	learn: 0.4215836	total: 12.8s	remaining: 48.2s
210:	learn: 0.4214315	total: 12.9s	remaining: 48.1s
211:	learn: 0.4213188	total: 12.9s	remaining: 48.1s
212:	learn: 0.4212125	total: 13s	remaining: 48s
213:	learn: 0.4210508	total: 13.1s	remaining: 47.9s
214:	learn: 0.4209128	total: 13.1s	remaining: 47.9s
215:	learn: 0.4207793	total: 13.2s	remaining: 47.8s
216:	learn: 0.4206380	total: 13.2s	remaining: 47.7s
217:	learn: 0.4205036	total: 13.3s	remaining: 47.7s
218:	learn: 0.4203790	total: 13.3s	remaining: 47.6s
219:	learn: 0.4202582	total: 13.4s	remaining: 47.5s
220:	learn: 0.4201363	total: 13.5s	remaining: 47.5s
221:	learn: 0.4200237	total: 13.5s	remaining: 47.4s
222:	learn: 0.4199125	total: 13.6s	remaining: 47.3s
223:	learn: 0.4197711	total: 13.6s	remaining: 47.2s
224:	learn: 0.4196386	total: 13.7s	remaining: 47.2s
225:	learn: 0.4195252	total: 13.8s	remaining: 47.1s
226:	learn: 0.4194294	total: 13.8s	remaining: 47.1s
227:	learn: 0.41

367:	learn: 0.4085152	total: 22.4s	remaining: 38.5s
368:	learn: 0.4084657	total: 22.5s	remaining: 38.4s
369:	learn: 0.4084201	total: 22.5s	remaining: 38.3s
370:	learn: 0.4083547	total: 22.6s	remaining: 38.3s
371:	learn: 0.4082766	total: 22.6s	remaining: 38.2s
372:	learn: 0.4082307	total: 22.7s	remaining: 38.1s
373:	learn: 0.4081872	total: 22.7s	remaining: 38.1s
374:	learn: 0.4081345	total: 22.8s	remaining: 38s
375:	learn: 0.4080871	total: 22.9s	remaining: 37.9s
376:	learn: 0.4080471	total: 22.9s	remaining: 37.9s
377:	learn: 0.4079944	total: 23s	remaining: 37.8s
378:	learn: 0.4079317	total: 23s	remaining: 37.7s
379:	learn: 0.4078900	total: 23.1s	remaining: 37.7s
380:	learn: 0.4078353	total: 23.1s	remaining: 37.6s
381:	learn: 0.4077850	total: 23.2s	remaining: 37.5s
382:	learn: 0.4077479	total: 23.3s	remaining: 37.5s
383:	learn: 0.4077011	total: 23.3s	remaining: 37.4s
384:	learn: 0.4076502	total: 23.4s	remaining: 37.3s
385:	learn: 0.4076010	total: 23.4s	remaining: 37.3s
386:	learn: 0.4075

527:	learn: 0.4020589	total: 31.8s	remaining: 28.4s
528:	learn: 0.4020232	total: 31.9s	remaining: 28.4s
529:	learn: 0.4019915	total: 32s	remaining: 28.4s
530:	learn: 0.4019613	total: 32s	remaining: 28.3s
531:	learn: 0.4019272	total: 32.1s	remaining: 28.2s
532:	learn: 0.4018947	total: 32.2s	remaining: 28.2s
533:	learn: 0.4018567	total: 32.2s	remaining: 28.1s
534:	learn: 0.4018202	total: 32.3s	remaining: 28.1s
535:	learn: 0.4017976	total: 32.4s	remaining: 28s
536:	learn: 0.4017550	total: 32.4s	remaining: 28s
537:	learn: 0.4017178	total: 32.5s	remaining: 27.9s
538:	learn: 0.4016778	total: 32.6s	remaining: 27.9s
539:	learn: 0.4016507	total: 32.6s	remaining: 27.8s
540:	learn: 0.4016172	total: 32.7s	remaining: 27.8s
541:	learn: 0.4015853	total: 32.8s	remaining: 27.7s
542:	learn: 0.4015443	total: 32.8s	remaining: 27.6s
543:	learn: 0.4015165	total: 32.9s	remaining: 27.6s
544:	learn: 0.4014902	total: 33s	remaining: 27.5s
545:	learn: 0.4014569	total: 33s	remaining: 27.5s
546:	learn: 0.4014224	to

688:	learn: 0.3972792	total: 41.5s	remaining: 18.7s
689:	learn: 0.3972465	total: 41.5s	remaining: 18.7s
690:	learn: 0.3972123	total: 41.6s	remaining: 18.6s
691:	learn: 0.3971952	total: 41.7s	remaining: 18.5s
692:	learn: 0.3971746	total: 41.7s	remaining: 18.5s
693:	learn: 0.3971443	total: 41.8s	remaining: 18.4s
694:	learn: 0.3971072	total: 41.8s	remaining: 18.4s
695:	learn: 0.3970756	total: 41.9s	remaining: 18.3s
696:	learn: 0.3970364	total: 41.9s	remaining: 18.2s
697:	learn: 0.3970140	total: 42s	remaining: 18.2s
698:	learn: 0.3969871	total: 42.1s	remaining: 18.1s
699:	learn: 0.3969593	total: 42.1s	remaining: 18.1s
700:	learn: 0.3969334	total: 42.2s	remaining: 18s
701:	learn: 0.3969088	total: 42.2s	remaining: 17.9s
702:	learn: 0.3968757	total: 42.3s	remaining: 17.9s
703:	learn: 0.3968367	total: 42.4s	remaining: 17.8s
704:	learn: 0.3968126	total: 42.4s	remaining: 17.7s
705:	learn: 0.3967814	total: 42.5s	remaining: 17.7s
706:	learn: 0.3967530	total: 42.5s	remaining: 17.6s
707:	learn: 0.39

850:	learn: 0.3931333	total: 51s	remaining: 8.94s
851:	learn: 0.3931046	total: 51.1s	remaining: 8.88s
852:	learn: 0.3930800	total: 51.2s	remaining: 8.81s
853:	learn: 0.3930612	total: 51.2s	remaining: 8.76s
854:	learn: 0.3930399	total: 51.3s	remaining: 8.69s
855:	learn: 0.3930137	total: 51.3s	remaining: 8.63s
856:	learn: 0.3929850	total: 51.4s	remaining: 8.57s
857:	learn: 0.3929720	total: 51.4s	remaining: 8.51s
858:	learn: 0.3929508	total: 51.5s	remaining: 8.45s
859:	learn: 0.3929399	total: 51.6s	remaining: 8.39s
860:	learn: 0.3929130	total: 51.6s	remaining: 8.33s
861:	learn: 0.3928900	total: 51.7s	remaining: 8.27s
862:	learn: 0.3928596	total: 51.7s	remaining: 8.21s
863:	learn: 0.3928372	total: 51.8s	remaining: 8.15s
864:	learn: 0.3928213	total: 51.8s	remaining: 8.09s
865:	learn: 0.3928014	total: 51.9s	remaining: 8.03s
866:	learn: 0.3927665	total: 52s	remaining: 7.97s
867:	learn: 0.3927448	total: 52s	remaining: 7.91s
868:	learn: 0.3927101	total: 52.1s	remaining: 7.85s
869:	learn: 0.3926

In [31]:
def scaled_model(model, X, y):
    strategies = ['mean', 'median', 'most_frequent', 'constant']
    for strategy in strategies:
        imputer = SimpleImputer(strategy = strategy)
        scaler = StandardScaler()
        power = PowerTransformer(method = 'yeo-johnson', standardize = True)
        
        steps = [('imputer', imputer),
                 ('scaler', scaler),
                 ('power', power),
                 ('model', model)]
        
        pipeline = Pipeline(steps = steps)
        scores, std = evaluate_model(pipeline, X, y)
        
        print('{} {} Accuracy: {}, std: {}'.format(model, strategy.capitalize(), scores, std))

In [32]:
def unscaled_model(model, X, y):
    strategies = ['mean', 'median', 'most_frequent', 'constant']
    for strategy in strategies:
        imputer = SimpleImputer(strategy = strategy)
        scaler = RobustScaler()
        power = PowerTransformer(method = 'yeo-johnson', standardize = True)
        
        steps = [('imputer', imputer),
                 ('scaler', scaler),
                 ('power', power),
                 ('model', model)]
        
        pipeline = Pipeline(steps = steps)
        scores, std = evaluate_model(pipeline, X, y)
        
        print('{} {} Accuracy: {}, std: {}'.format(model, strategy.capitalize(), scores, std))

In [37]:
# Creating pipeline and evaluating model for AdaBoost

scaled_model(modelA, X, y)

AdaBoostClassifier(random_state=1) Mean Accuracy: 82.79954178091013, std: 0.6562187550245208
AdaBoostClassifier(random_state=1) Median Accuracy: 82.8177317333102, std: 0.711666867443442
AdaBoostClassifier(random_state=1) Most_frequent Accuracy: 82.67567058793301, std: 0.6903854601486842
AdaBoostClassifier(random_state=1) Constant Accuracy: 82.83080201597848, std: 0.6636710227794438


In [ ]:
# Creating pipeline and evaluating model for AdaBoost (unscaled data)

unscaled_model(modelA, X, y)

In [24]:
# Creating pipeline and evaluating model for RFC

scaled_model(modelB, X, y)

RandomForestClassifier(criterion='entropy', random_state=1) Mean Accuracy: 83.25432894364849, std: 0.6553163154395325
RandomForestClassifier(criterion='entropy', random_state=1) Median Accuracy: 83.24425234142471, std: 0.7482122344567416
RandomForestClassifier(criterion='entropy', random_state=1) Most_frequent Accuracy: 83.2837617078452, std: 0.718847752043315
RandomForestClassifier(criterion='entropy', random_state=1) Constant Accuracy: 83.28146142632392, std: 0.6721365244905828


In [25]:
# Creating pipeline and evaluating model for RFC (unscaled data)

unscaled_model(modelB, X, y)

RandomForestClassifier(criterion='entropy', random_state=1) Mean Accuracy: 83.26269472497685, std: 0.7205185767848161
RandomForestClassifier(criterion='entropy', random_state=1) Median Accuracy: 83.2125754028374, std: 0.703765569357522
RandomForestClassifier(criterion='entropy', random_state=1) Most_frequent Accuracy: 83.27026034854224, std: 0.7036338386745228
RandomForestClassifier(criterion='entropy', random_state=1) Constant Accuracy: 83.2464887018557, std: 0.6980248015618805


In [26]:
# Creating pipeline and evaluating model for ExtraTreesClassifier

scaled_model(modelC, X, y)

ExtraTreesClassifier(criterion='entropy', random_state=1) Mean Accuracy: 83.00798309585694, std: 0.6872156559516928
ExtraTreesClassifier(criterion='entropy', random_state=1) Median Accuracy: 82.97549702581564, std: 0.7103411424200217
ExtraTreesClassifier(criterion='entropy', random_state=1) Most_frequent Accuracy: 83.01790174267225, std: 0.6980048033594202
ExtraTreesClassifier(criterion='entropy', random_state=1) Constant Accuracy: 83.06211421269555, std: 0.7180927355595889


In [27]:
# Creating pipeline and evaluating model for ExtraTreesClassifier (unscaled data)

unscaled_model(modelC, X, y)

ExtraTreesClassifier(criterion='entropy', random_state=1) Mean Accuracy: 82.97332134839705, std: 0.6641763423030389
ExtraTreesClassifier(criterion='entropy', random_state=1) Median Accuracy: 83.05615650186559, std: 0.7256379301354137
ExtraTreesClassifier(criterion='entropy', random_state=1) Most_frequent Accuracy: 83.04969151801048, std: 0.7717199403234257
ExtraTreesClassifier(criterion='entropy', random_state=1) Constant Accuracy: 83.09100156149094, std: 0.7202644747379302


In [28]:
# Creating pipeline and evaluating model for LGBM

scaled_model(modelD, X, y)

LGBMClassifier(random_state=1) Mean Accuracy: 83.69614669418311, std: 0.625202418985817
LGBMClassifier(random_state=1) Median Accuracy: 83.63134512795264, std: 0.6794749970880637
LGBMClassifier(random_state=1) Most_frequent Accuracy: 83.65418498995922, std: 0.7039529602422804
LGBMClassifier(random_state=1) Constant Accuracy: 83.74415252711536, std: 0.6640624777991867


In [29]:
# Creating pipeline and evaluating model for LGBM (unscaled data)

scaled_model(modelD, X, y)

LGBMClassifier(random_state=1) Mean Accuracy: 83.70988264988894, std: 0.6631803592997823
LGBMClassifier(random_state=1) Median Accuracy: 83.64108240485628, std: 0.6595503099646315
LGBMClassifier(random_state=1) Most_frequent Accuracy: 83.62121672178759, std: 0.6745685834477981
LGBMClassifier(random_state=1) Constant Accuracy: 83.71798622199432, std: 0.6594814820455736


# Muy importante...

In [ ]:
# Creating pipeline and evaluating model for CatBoost

scaled_model(modelE, X, y)

<catboost.core.CatBoostClassifier object at 0x000002355F02B760> Mean Accuracy: 84.03128055486772, std: 0.6347417931248309


In [ ]:
# Creating pipeline and evaluating model for CatBoost (unscaled)

unscaled_model(modelE, X, y)

In [ ]:
# Creating pipeline and evaluating model for GradientBM

scaled_model(modelF, X, y)

In [ ]:
# Creating pipeline and evaluating model for GradientBM (unscaled)

unscaled_model(modelF, X, y)

Apparently, scaling the data through standardization techniques has very little benefits when training the model.

Apparently, scaling the data does not give much change in accuracy for the LGBM algorithm.

Standardization does not provide much change in accuracy for the ExtraTreesClassifier. The constant strategy seems to produce the highest accuracy irrespective of standard scaling.

# The RandomForest, AdaBoost, LGBM and ExtraTrees models will be fitted using the scaled data and the constant imputer strategy, after which they will be combined through a voting classifier.

In [58]:
def prep_pipe(model):
    
    """Returns a model pipeline."""
    
    imputer = SimpleImputer(strategy = 'constant')
    scaler = RobustScaler()
    power = PowerTransformer(method = 'yeo-johnson',
                             standardize = True)
    steps = [('imputer', imputer),
             ('scaler', scaler),
             ('power', power),
             ('model', model)]
    
    pipeline = Pipeline(steps = steps)
    return pipeline

In [34]:
def prep_test(test):
    
    """Returns Applicant ID and prepared test data."""
    
    ID = test['Applicant_ID']
    test_p = test.drop(['Applicant_ID'], axis = 1).values
    
    return ID, test_p

In [35]:
def final_model(model, X, y, test):
    
    """Fits estimation pipeline, and returns the applicant IDs, predictions, score, and standard deviation."""
    pipeline = prep_pipe(model)
    pipeline.fit(X, y)
    
    ID, test_p = prep_test(test)
    ypred = pipeline.predict_proba(test_p)
    score, std = evaluate_model(pipeline, X, y)
    
    return ID, ypred, score, std

# Preparing the prediction pipelines...

In [59]:
# Fitting the AdaBoost pipeline with scaled data for the constant stategy

pipelineA = prep_pipe(modelA)

In [60]:
# Fitting the RandomForest pipeline with scaled data for the constant stategy

pipelineB = prep_pipe(modelB)

In [61]:
# Fitting the ExtraTrees pipeline with scaled data for the constant stategy

pipelineC = prep_pipe(modelC)

In [62]:
# Fitting the LGBM pipeline with scaled data for the constant stategy

pipelineD = prep_pipe(modelD)

In [63]:
# Fitting the CatBoost pipeline with scaled data for the constant stategy

pipelineE = prep_pipe(modelE)

In [64]:
# Fitting the GradientBoostingClassifier pipeline with scaled data for the constant stategy

pipelineF = prep_pipe(modelF)

# Fitting CatBoost pipe alone...

In [65]:
pipelineE.fit(X, y)
ycat = pipelineE.predict_proba(prep_test(test)[1])

0:	learn: 0.6863364	total: 110ms	remaining: 3m 6s
1:	learn: 0.6798106	total: 211ms	remaining: 2m 58s
2:	learn: 0.6735739	total: 315ms	remaining: 2m 58s
3:	learn: 0.6673784	total: 430ms	remaining: 3m 2s
4:	learn: 0.6609481	total: 564ms	remaining: 3m 11s
5:	learn: 0.6551450	total: 668ms	remaining: 3m 8s
6:	learn: 0.6494606	total: 787ms	remaining: 3m 10s
7:	learn: 0.6437144	total: 898ms	remaining: 3m 10s
8:	learn: 0.6382612	total: 1.01s	remaining: 3m 9s
9:	learn: 0.6326463	total: 1.12s	remaining: 3m 9s
10:	learn: 0.6275222	total: 1.25s	remaining: 3m 12s
11:	learn: 0.6223740	total: 1.35s	remaining: 3m 10s
12:	learn: 0.6175946	total: 1.46s	remaining: 3m 9s
13:	learn: 0.6124518	total: 1.57s	remaining: 3m 8s
14:	learn: 0.6077269	total: 1.67s	remaining: 3m 7s
15:	learn: 0.6031513	total: 1.77s	remaining: 3m 6s
16:	learn: 0.5986521	total: 1.88s	remaining: 3m 6s
17:	learn: 0.5942508	total: 1.99s	remaining: 3m 6s
18:	learn: 0.5900410	total: 2.13s	remaining: 3m 8s
19:	learn: 0.5859157	total: 2.23s	

159:	learn: 0.4225998	total: 19.8s	remaining: 3m 10s
160:	learn: 0.4223368	total: 19.9s	remaining: 3m 10s
161:	learn: 0.4221038	total: 20s	remaining: 3m 10s
162:	learn: 0.4218693	total: 20.2s	remaining: 3m 10s
163:	learn: 0.4216639	total: 20.3s	remaining: 3m 9s
164:	learn: 0.4214506	total: 20.4s	remaining: 3m 9s
165:	learn: 0.4212127	total: 20.5s	remaining: 3m 9s
166:	learn: 0.4209695	total: 20.6s	remaining: 3m 9s
167:	learn: 0.4207500	total: 20.7s	remaining: 3m 9s
168:	learn: 0.4205264	total: 20.9s	remaining: 3m 8s
169:	learn: 0.4202818	total: 21s	remaining: 3m 8s
170:	learn: 0.4200325	total: 21.1s	remaining: 3m 8s
171:	learn: 0.4198125	total: 21.2s	remaining: 3m 8s
172:	learn: 0.4195910	total: 21.3s	remaining: 3m 7s
173:	learn: 0.4193990	total: 21.4s	remaining: 3m 7s
174:	learn: 0.4191929	total: 21.5s	remaining: 3m 7s
175:	learn: 0.4189843	total: 21.6s	remaining: 3m 7s
176:	learn: 0.4188144	total: 21.7s	remaining: 3m 6s
177:	learn: 0.4186189	total: 21.8s	remaining: 3m 6s
178:	learn: 

316:	learn: 0.4027874	total: 37s	remaining: 2m 41s
317:	learn: 0.4027221	total: 37.2s	remaining: 2m 41s
318:	learn: 0.4026584	total: 37.3s	remaining: 2m 41s
319:	learn: 0.4026017	total: 37.4s	remaining: 2m 41s
320:	learn: 0.4025305	total: 37.5s	remaining: 2m 41s
321:	learn: 0.4024580	total: 37.6s	remaining: 2m 40s
322:	learn: 0.4023958	total: 37.7s	remaining: 2m 40s
323:	learn: 0.4023260	total: 37.9s	remaining: 2m 40s
324:	learn: 0.4022649	total: 38s	remaining: 2m 40s
325:	learn: 0.4022043	total: 38.1s	remaining: 2m 40s
326:	learn: 0.4021168	total: 38.2s	remaining: 2m 40s
327:	learn: 0.4020729	total: 38.3s	remaining: 2m 40s
328:	learn: 0.4020023	total: 38.4s	remaining: 2m 40s
329:	learn: 0.4019400	total: 38.5s	remaining: 2m 39s
330:	learn: 0.4018707	total: 38.6s	remaining: 2m 39s
331:	learn: 0.4018005	total: 38.8s	remaining: 2m 39s
332:	learn: 0.4017407	total: 38.9s	remaining: 2m 39s
333:	learn: 0.4016753	total: 39s	remaining: 2m 39s
334:	learn: 0.4016034	total: 39.1s	remaining: 2m 39s

472:	learn: 0.3942693	total: 54.1s	remaining: 2m 20s
473:	learn: 0.3942138	total: 54.2s	remaining: 2m 20s
474:	learn: 0.3941541	total: 54.3s	remaining: 2m 20s
475:	learn: 0.3941081	total: 54.4s	remaining: 2m 20s
476:	learn: 0.3940670	total: 54.5s	remaining: 2m 19s
477:	learn: 0.3940161	total: 54.7s	remaining: 2m 19s
478:	learn: 0.3939698	total: 54.8s	remaining: 2m 19s
479:	learn: 0.3939301	total: 54.9s	remaining: 2m 19s
480:	learn: 0.3938908	total: 55s	remaining: 2m 19s
481:	learn: 0.3938502	total: 55.1s	remaining: 2m 19s
482:	learn: 0.3937880	total: 55.2s	remaining: 2m 19s
483:	learn: 0.3937547	total: 55.4s	remaining: 2m 19s
484:	learn: 0.3937065	total: 55.5s	remaining: 2m 18s
485:	learn: 0.3936556	total: 55.6s	remaining: 2m 18s
486:	learn: 0.3936126	total: 55.7s	remaining: 2m 18s
487:	learn: 0.3935638	total: 55.8s	remaining: 2m 18s
488:	learn: 0.3935225	total: 55.9s	remaining: 2m 18s
489:	learn: 0.3934892	total: 56s	remaining: 2m 18s
490:	learn: 0.3934221	total: 56.1s	remaining: 2m 1

630:	learn: 0.3877002	total: 1m 11s	remaining: 2m
631:	learn: 0.3876561	total: 1m 11s	remaining: 2m
632:	learn: 0.3876291	total: 1m 11s	remaining: 2m
633:	learn: 0.3875994	total: 1m 11s	remaining: 2m
634:	learn: 0.3875566	total: 1m 11s	remaining: 2m
635:	learn: 0.3875206	total: 1m 11s	remaining: 2m
636:	learn: 0.3874780	total: 1m 12s	remaining: 2m
637:	learn: 0.3874425	total: 1m 12s	remaining: 2m
638:	learn: 0.3874144	total: 1m 12s	remaining: 1m 59s
639:	learn: 0.3873553	total: 1m 12s	remaining: 1m 59s
640:	learn: 0.3873220	total: 1m 12s	remaining: 1m 59s
641:	learn: 0.3872909	total: 1m 12s	remaining: 1m 59s
642:	learn: 0.3872748	total: 1m 12s	remaining: 1m 59s
643:	learn: 0.3872384	total: 1m 12s	remaining: 1m 59s
644:	learn: 0.3872018	total: 1m 12s	remaining: 1m 59s
645:	learn: 0.3871599	total: 1m 13s	remaining: 1m 59s
646:	learn: 0.3871181	total: 1m 13s	remaining: 1m 59s
647:	learn: 0.3870855	total: 1m 13s	remaining: 1m 58s
648:	learn: 0.3870486	total: 1m 13s	remaining: 1m 58s
649:	l

...

937:	learn: 0.3774558	total: 1m 45s	remaining: 1m 25s
938:	learn: 0.3774234	total: 1m 45s	remaining: 1m 25s
939:	learn: 0.3773848	total: 1m 45s	remaining: 1m 25s
940:	learn: 0.3773469	total: 1m 46s	remaining: 1m 25s
941:	learn: 0.3773175	total: 1m 46s	remaining: 1m 25s
942:	learn: 0.3772843	total: 1m 46s	remaining: 1m 25s
943:	learn: 0.3772566	total: 1m 46s	remaining: 1m 25s
944:	learn: 0.3772227	total: 1m 46s	remaining: 1m 25s
945:	learn: 0.3771811	total: 1m 46s	remaining: 1m 24s
946:	learn: 0.3771485	total: 1m 46s	remaining: 1m 24s
947:	learn: 0.3771203	total: 1m 46s	remaining: 1m 24s
948:	learn: 0.3770881	total: 1m 46s	remaining: 1m 24s
949:	learn: 0.3770625	total: 1m 47s	remaining: 1m 24s
950:	learn: 0.3770239	total: 1m 47s	remaining: 1m 24s
951:	learn: 0.3769954	total: 1m 47s	remaining: 1m 24s
952:	learn: 0.3769940	total: 1m 47s	remaining: 1m 24s
953:	learn: 0.3769583	total: 1m 47s	remaining: 1m 23s
954:	learn: 0.3769332	total: 1m 47s	remaining: 1m 23s
955:	learn: 0.3769091	total:

1089:	learn: 0.3723775	total: 2m 2s	remaining: 1m 8s
1090:	learn: 0.3723432	total: 2m 2s	remaining: 1m 8s
1091:	learn: 0.3723076	total: 2m 2s	remaining: 1m 8s
1092:	learn: 0.3722779	total: 2m 2s	remaining: 1m 8s
1093:	learn: 0.3722451	total: 2m 2s	remaining: 1m 7s
1094:	learn: 0.3722179	total: 2m 2s	remaining: 1m 7s
1095:	learn: 0.3721898	total: 2m 2s	remaining: 1m 7s
1096:	learn: 0.3721573	total: 2m 3s	remaining: 1m 7s
1097:	learn: 0.3721209	total: 2m 3s	remaining: 1m 7s
1098:	learn: 0.3720776	total: 2m 3s	remaining: 1m 7s
1099:	learn: 0.3720518	total: 2m 3s	remaining: 1m 7s
1100:	learn: 0.3720225	total: 2m 3s	remaining: 1m 7s
1101:	learn: 0.3719854	total: 2m 3s	remaining: 1m 7s
1102:	learn: 0.3719583	total: 2m 3s	remaining: 1m 6s
1103:	learn: 0.3719269	total: 2m 3s	remaining: 1m 6s
1104:	learn: 0.3718973	total: 2m 3s	remaining: 1m 6s
1105:	learn: 0.3718507	total: 2m 4s	remaining: 1m 6s
1106:	learn: 0.3718261	total: 2m 4s	remaining: 1m 6s
1107:	learn: 0.3717923	total: 2m 4s	remaining:

1243:	learn: 0.3669841	total: 2m 20s	remaining: 51.3s
1244:	learn: 0.3669416	total: 2m 20s	remaining: 51.2s
1245:	learn: 0.3669105	total: 2m 20s	remaining: 51.1s
1246:	learn: 0.3668896	total: 2m 20s	remaining: 51s
1247:	learn: 0.3668582	total: 2m 20s	remaining: 50.9s
1248:	learn: 0.3668166	total: 2m 20s	remaining: 50.8s
1249:	learn: 0.3667764	total: 2m 20s	remaining: 50.7s
1250:	learn: 0.3667466	total: 2m 20s	remaining: 50.6s
1251:	learn: 0.3667104	total: 2m 21s	remaining: 50.5s
1252:	learn: 0.3666729	total: 2m 21s	remaining: 50.4s
1253:	learn: 0.3666432	total: 2m 21s	remaining: 50.2s
1254:	learn: 0.3666164	total: 2m 21s	remaining: 50.1s
1255:	learn: 0.3665852	total: 2m 21s	remaining: 50s
1256:	learn: 0.3665487	total: 2m 21s	remaining: 49.9s
1257:	learn: 0.3665210	total: 2m 21s	remaining: 49.8s
1258:	learn: 0.3664784	total: 2m 21s	remaining: 49.7s
1259:	learn: 0.3664457	total: 2m 22s	remaining: 49.6s
1260:	learn: 0.3664142	total: 2m 22s	remaining: 49.5s
1261:	learn: 0.3663786	total: 2m

1396:	learn: 0.3616813	total: 2m 38s	remaining: 34.4s
1397:	learn: 0.3616537	total: 2m 38s	remaining: 34.2s
1398:	learn: 0.3616139	total: 2m 38s	remaining: 34.1s
1399:	learn: 0.3615818	total: 2m 38s	remaining: 34s
1400:	learn: 0.3615597	total: 2m 38s	remaining: 33.9s
1401:	learn: 0.3615362	total: 2m 38s	remaining: 33.8s
1402:	learn: 0.3615003	total: 2m 39s	remaining: 33.7s
1403:	learn: 0.3614607	total: 2m 39s	remaining: 33.6s
1404:	learn: 0.3614219	total: 2m 39s	remaining: 33.4s
1405:	learn: 0.3613851	total: 2m 39s	remaining: 33.3s
1406:	learn: 0.3613433	total: 2m 39s	remaining: 33.2s
1407:	learn: 0.3613040	total: 2m 39s	remaining: 33.1s
1408:	learn: 0.3612633	total: 2m 39s	remaining: 33s
1409:	learn: 0.3612289	total: 2m 39s	remaining: 32.9s
1410:	learn: 0.3611867	total: 2m 40s	remaining: 32.8s
1411:	learn: 0.3611481	total: 2m 40s	remaining: 32.7s
1412:	learn: 0.3611062	total: 2m 40s	remaining: 32.6s
1413:	learn: 0.3610619	total: 2m 40s	remaining: 32.4s
1414:	learn: 0.3610300	total: 2m

1550:	learn: 0.3566040	total: 2m 56s	remaining: 16.9s
1551:	learn: 0.3565688	total: 2m 56s	remaining: 16.8s
1552:	learn: 0.3565324	total: 2m 56s	remaining: 16.7s
1553:	learn: 0.3564939	total: 2m 57s	remaining: 16.6s
1554:	learn: 0.3564564	total: 2m 57s	remaining: 16.5s
1555:	learn: 0.3564273	total: 2m 57s	remaining: 16.4s
1556:	learn: 0.3563955	total: 2m 57s	remaining: 16.3s
1557:	learn: 0.3563675	total: 2m 57s	remaining: 16.2s
1558:	learn: 0.3563460	total: 2m 57s	remaining: 16.1s
1559:	learn: 0.3563064	total: 2m 57s	remaining: 16s
1560:	learn: 0.3562683	total: 2m 58s	remaining: 15.9s
1561:	learn: 0.3562264	total: 2m 58s	remaining: 15.7s
1562:	learn: 0.3561979	total: 2m 58s	remaining: 15.6s
1563:	learn: 0.3561694	total: 2m 58s	remaining: 15.5s
1564:	learn: 0.3561398	total: 2m 58s	remaining: 15.4s
1565:	learn: 0.3561041	total: 2m 58s	remaining: 15.3s
1566:	learn: 0.3560657	total: 2m 58s	remaining: 15.2s
1567:	learn: 0.3560283	total: 2m 58s	remaining: 15.1s
1568:	learn: 0.3559916	total: 

In [ ]:
# Fiiting, predicting and evaluating CatBoost pipeline

ID, ycat, score, std = final_model(pipelineE, X, y, test)

In [66]:
df = pd.DataFrame({'Applicant_ID' : prep_test(test)[0],
                   'default_status' : ycat[:, 1]})

In [67]:
df.to_csv('SingleCAT(2).csv', index = False)

# Fitting a whole pipeline...

In [68]:
model_pipeA = [('Adaboost', pipelineA),
               ('RFC', pipelineB),
               ('XTC', pipelineC),
               ('LGBM', pipelineD),
               ('CatBoost', pipelineE)]
               #('GradientBM', pipelineF)]

In [69]:
electionS = VotingClassifier(estimators = model_pipeA, n_jobs = -1, voting = 'soft', verbose = True)

In [70]:
electionS.fit(X, y)

VotingClassifier(estimators=[('Adaboost',
                              Pipeline(steps=[('imputer',
                                               SimpleImputer(strategy='constant')),
                                              ('scaler', RobustScaler()),
                                              ('power', PowerTransformer()),
                                              ('model',
                                               AdaBoostClassifier(random_state=1))])),
                             ('RFC',
                              Pipeline(steps=[('imputer',
                                               SimpleImputer(strategy='constant')),
                                              ('scaler', RobustScaler()),
                                              ('power', PowerTransformer()),
                                              ('model',
                                               RandomFor...
                                              ('power', PowerTransformer()),

In [ ]:
mean_roc_auc, mean_std = evaluate_model(electionS, X, y)

In [37]:
print('electionS roc_auc : {}, electionS std : {}'.format(mean_roc_auc, mean_std ))

electionS roc_auc : 83.97886019378473, electionS std : 0.6736057498676604


In [71]:
# For soft voting classification
yhat = electionS.predict_proba(prep_test(test)[1])

In [72]:
dfS = pd.DataFrame({'Applicant_ID' : prep_test(test)[0],
                    'default_status' : yhat[:, 1]})

In [73]:
Submission = dfS.to_csv('Submission12SCopy8(3).csv', index = False)

# Using Gaussian Mixture model...

In [57]:
cov_types = ['spherical', 'diag', 'tied', 'full']

for cov_type in cov_types:
    estimator = GaussianMixture(n_components = 2,
                                covariance_type = cov_type,
                                max_iter = 20,
                                random_state = 1)
    
    estimator.means_ = np.array([X[y == 1].mean(axis = 0) for i in range(2)])
    pipe = Pipeline([('imputer', SimpleImputer(strategy = 'constant')),
                     ('scaler', StandardScaler()),
                     ('power', PowerTransformer(method = 'yeo-johnson')),
                     ('estimator', estimator)])
    
    score, std = evaluate_model(estimator, X, y)
    print('Mean roc-auc: {}, Mean_std: {}'.format(score, std))

Mean roc-auc: nan, Mean_std: nan
Mean roc-auc: nan, Mean_std: nan
Mean roc-auc: nan, Mean_std: nan
Mean roc-auc: nan, Mean_std: nan
